# Geochronology Calculations

In [1]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from bokeh.models import Range1d, LinearAxis, ColumnDataSource, LabelSet, Span, Slope, Label, Legend

from scipy.interpolate import CubicSpline
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
output_notebook()

import geochron_apps as gc

Loading BokehJS ...

<center><img src="images/geochronology.png" align="center">
https://www.explainxkcd.com/wiki/index.php/1829:_Geochronology
</center>

The following presentation shows some of the geochronology calculations learned in the Advanced Geochronology class at University of Saskatchewan taught by Bruce Eglington and Camille Partin, 2021. Some of the images in this presentation are taken from lectures given by the instructor.

This notebook contains sample calculations typically used in geochronology. It can be obtained at https://git.cs.usask.ca/msv275/advanced-geochronology.  
It can be cloned through the git command:  
* git clone https://git.cs.usask.ca/msv275/advanced-geochronology.git


# Lf-Hf Calculations

Start with an appropriate value for depleted mantle at 4570 Ma and calculate and graph the curve for depleted mantle.

Our variables:  
* Decay Constant = 1.867 x 10<sup>-11</sup> (Scherer, Munker, and Mezger 2001)  
* <sup>176</sup>Lu/<sup>177</sup>Hf<sub>(depleted mantle)</sub> = 0.0384 (Chauvel and Blichert-Toft 2001)  
* <sup>176</sup>Hf/<sup>177</sup>Hf<sub>(depleted mantle)</sub> = 0.283250 (Chauvel and Blichert-Toft 2001)  

Isochron Equation:

<sup>176</sup>Hf/<sup>177</sup>Hf<sub>(present day)</sub> = <sup>176</sup>Hf/<sup>177</sup>Hf<sub>(initial)</sub> + <sup>176</sup>Lu/<sup>177</sup>Hf<sub>(present day)</sub> * (*e*<sup>λ x t</sup> - 1)


In [2]:
decay_const_177 = 1.867 * 10 ** -11 
Lu_DM = 0.0384
Hf_DM = 0.283250
Lu_CHUR = 0.0336
Hf_CHUR = 0.282785

d = []
t1, t2 = 4570, 0
while t1 > 0:
    d.append({'t1': t1,
            't2': t2,
            '176Lu/177Hf': Lu_DM, 
            '176Hf/177Hf_DM': gc.calc_initial(Hf_DM, Lu_DM, decay_const_177, t1, t2),
            '176Hf/177Hf_CHUR': gc.calc_initial(Hf_CHUR, Lu_CHUR, decay_const_177, t1, t2),
                 })
    t1 = t1 - 1
LuHf_df = pd.DataFrame(d)
LuHf_df.head()

,t1,t2,176Lu/177Hf,176Hf/177Hf_DM,176Hf/177Hf_CHUR
0,4570,0,0.0384,0.279830,0.279792
1,4569,0,0.0384,0.279831,0.279793
2,4568,0,0.0384,0.279831,0.279794
3,4567,0,0.0384,0.279832,0.279794
4,4566,0,0.0384,0.279833,0.279795


In [3]:
figure11 = gc.get_figure("176Hf/177Hf", "176Hf/177Hf", "Age (Ma)", [0,4570], [0.279,0.29])
figure11.line(LuHf_df['t1'], LuHf_df['176Hf/177Hf_DM'], color="darkred", legend_label="Depleted Mantle")

figure11.legend.location = "top_right"
figure11.legend.click_policy="hide"

In [4]:
show(figure11)

Assume a crust generation event at 3000 Ma and another at 500 Ma, each starting from the depleted mantle curve. Assume these produce felsic crust with 176Lu/177Hf values of 0.15. Calculate and graph these two curves plus the curve for CHUR and for depleted mantle. 

In [22]:
c_event1 = 3000
c_event2 = 500
LuHf_event = 0.15
Hf_DM_3000 = LuHf_df[LuHf_df['t1'] == 3000].values[0][-2]
Hf_DM_500 = LuHf_df[LuHf_df['t1'] == 500].values[0][-2]
Hf_CHUR_3000 = LuHf_df[LuHf_df['t1'] == 3000].values[0][-1]
Hf_CHUR_500 = LuHf_df[LuHf_df['t1'] == 500].values[0][-1]

d = []
t1, t2 = 3000, 0
while t2 < 3000:
    d.append({'t1': t1,
            't2': t2,
            '176Lu/177Hf': Lu_DM, 
            '176Hf/177Hf_DM': gc.calc_t2_daughter(Hf_DM_3000, LuHf_event, decay_const_177, t1, t2),
            '176Hf/177Hf_CHUR': gc.calc_t2_daughter(Hf_CHUR_3000, LuHf_event, decay_const_177, t1, t2),
                 })
    t2 = t2 + 1

LuHf_3000_df = pd.DataFrame(d)

d = []
t1, t2 = 500, 0
while t2 < 500:
    d.append({'t1': t1,
            't2': t2,
            '176Lu/177Hf': Lu_DM, 
            '176Hf/177Hf_DM': gc.calc_t2_daughter(Hf_DM_500, LuHf_event, decay_const_177, t1, t2),
            '176Hf/177Hf_CHUR': gc.calc_t2_daughter(Hf_CHUR_500, LuHf_event, decay_const_177, t1, t2),
                 })
    t2 = t2 + 1

LuHf_500_df = pd.DataFrame(d)

In [23]:
figure12 = gc.get_figure("176Hf/177Hf", "176Hf/177Hf", "Age (Ma)", [0,4570], [0.279,0.29])
figure11.line(LuHf_df['t1'], LuHf_df['176Hf/177Hf_DM'], color="darkred", legend_label="Depleted Mantle")
figure12.line(LuHf_df['t1'], LuHf_df['176Hf/177Hf_CHUR'], color="darkblue", legend_label="CHUR")
figure12.line(LuHf_3000_df['t2'], LuHf_3000_df['176Hf/177Hf_CHUR'], color="lightblue", legend_label="3000 Ma Event (CHUR)")
figure12.line(LuHf_3000_df['t2'], LuHf_3000_df['176Hf/177Hf_DM'], color="pink", legend_label="3000 Ma Event (DM)")
figure12.line(LuHf_500_df['t2'], LuHf_500_df['176Hf/177Hf_CHUR'], color="blue", legend_label="500 Ma Event (CHUR)")
figure12.line(LuHf_500_df['t2'], LuHf_500_df['176Hf/177Hf_DM'], color="red", legend_label="500 Ma Event (DM)")

GlyphRenderer(id='4125', ...)

In [24]:
show(figure12)

Assume that the 3000 Ma crust melts at 1000 Ma to produce a felsic igneous rock which crystallizes to form zircon with a 176Lu/177Hf value of 0.00001. Calculate and graph the evolution of this zircon to the present day. 

In [8]:
c_event3 = 1000
LuHf_event = 0.00001
Hf_DM_1000 = LuHf_3000_df[LuHf_3000_df['t2'] == 1000].values[0][-2]
Hf_CHUR_1000 = LuHf_3000_df[LuHf_3000_df['t2'] == 1000].values[0][-1]


d = []
t1, t2 = 1000, 0
while t2 < 1000:
    d.append({'t1': t1,
            't2': t2,
            '176Hf/177Hf_DM': gc.calc_t2_daughter(Hf_DM_1000, LuHf_event, decay_const_177, t1, t2),
            '176Hf/177Hf_CHUR': gc.calc_t2_daughter(Hf_CHUR_1000, LuHf_event, decay_const_177, t1, t2),
                 })
    t2 = t2 + 1

LuHf_1000_df = pd.DataFrame(d)

In [9]:
figure11.line(LuHf_1000_df['t2'], LuHf_1000_df['176Hf/177Hf_CHUR'], color="lightblue", legend_label="1000 Ma Event (CHUR)")
figure11.line(LuHf_1000_df['t2'], LuHf_1000_df['176Hf/177Hf_DM'], color="pink", legend_label="1000 Ma Event (DM)")

GlyphRenderer(id='1500', ...)

In [10]:
show(figure11)

Assume that zircon from an igneous rock sample is analysed and provides the following composition:
176Lu/177Hf= 0.0003
176Hf/177Hf= 0.286000

The igneous unit has previously been dated at 500 Ma by U-Pb

Calculate the initial composition (ratio and epsilon value). What is its T(DM) and T(2DM)?

Assume the crustal evolution history used in the previous assignment and determine possible scenarios for mixing of multiple end-members to explain the composition of the zircon and host igneous rock sample at its time of formation. Assume that Hf concentrations in mantle and crust are 0.203 and 4.370, respectively and that Lu concentrations are 0.1 and 5.0, respectively.

Illustrate the scenarios graphically in Excel for both ratio and epsilon situations